In [11]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
# from utils import save_config_file, accuracy, save_checkpoint
# import torch.nn as nn

# import argparse
import torch
# import torch.backends.cudnn as cudnn
# from torchvision import models
# from data_aug.contrastive_learning_dataset import ContrastiveLearningDataset
from models.resnet_simclr import ResNetSimCLR
# from simclr import SimCLR
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset



In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [8]:
# load model
model = ResNetSimCLR(base_model='resnet18', out_dim=128)
model.load_state_dict(torch.load('/home/zixiwang/mtlsimclr/SimCLR/runs/Dec12_02-52-05_deeplearning-1-vm/checkpoint_0100.pth.tar', map_location=torch.device('cpu'))['state_dict'])
model = model.to(device)

In [12]:
# load adversarial images
path = '/home/zixiwang/mtlsimclr/SimCLR/cifar10_pgd.pt'
adv_images, adv_labels = torch.load(path)
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [15]:
# evluate  robustness result
model.eval()

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in adv_loader:
        images = images.to(device)
        labels = labels.to(device)
        prediction, features = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 10.00 %
